In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
# !rm multinli_1.0.zip

In [3]:
from glob import glob

files = sorted(glob('multinli_1.0/*.jsonl'))
files

['multinli_1.0/multinli_1.0_dev_matched.jsonl',
 'multinli_1.0/multinli_1.0_dev_mismatched.jsonl',
 'multinli_1.0/multinli_1.0_train.jsonl']

In [4]:
import json

data = []
with open(files[1]) as fopen:
    for l in fopen:
        data.append(json.loads(l))

len(data)

10000

In [5]:
data[0]

{'annotator_labels': ['contradiction',
  'contradiction',
  'contradiction',
  'contradiction',
  'contradiction'],
 'genre': 'letters',
 'gold_label': 'contradiction',
 'pairID': '75290c',
 'promptID': '75290',
 'sentence1': 'Your contribution helped make it possible for us to provide our students with a quality education.',
 'sentence1_binary_parse': '( ( Your contribution ) ( ( helped ( make ( it ( possible ( for ( us ( to ( ( provide ( our students ) ) ( with ( a ( quality education ) ) ) ) ) ) ) ) ) ) ) . ) )',
 'sentence1_parse': '(ROOT (S (NP (PRP$ Your) (NN contribution)) (VP (VBD helped) (VP (VB make) (S (NP (PRP it)) (ADJP (JJ possible)) (SBAR (IN for) (S (NP (PRP us)) (VP (TO to) (VP (VB provide) (NP (PRP$ our) (NNS students)) (PP (IN with) (NP (DT a) (NN quality) (NN education)))))))))) (. .)))',
 'sentence2': "Your contributions were of no help with our students' education.",
 'sentence2_binary_parse': "( ( Your contributions ) ( ( were ( of ( ( no help ) ( with ( ( our ( 

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('mesolitica/finetune-translation-t5-small-standard-bahasa-cased')
model = T5ForConditionalGeneration.from_pretrained('mesolitica/finetune-translation-t5-small-standard-bahasa-cased')

In [7]:
%%time

batch = [data[0]['sentence1'], data[0]['sentence2']]
t = ['terjemah Inggeris ke Melayu: ' + b for b in batch]
inputs = tokenizer(t, return_tensors="pt", padding = True)
translated_tokens = model.generate(**inputs, max_length=500)
decoded = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

CPU times: user 2.12 s, sys: 1.19 s, total: 3.3 s
Wall time: 1.67 s


In [8]:
from tqdm import tqdm

with open('translated-mnli-dev_mismatched.jsonl', 'w') as fopenl:
    for d in tqdm(data):
        batch = [d['sentence1'], d['sentence2']]
        t = ['terjemah Inggeris ke Melayu: ' + b for b in batch]
        inputs = tokenizer(t, return_tensors="pt", padding = True)
        translated_tokens = model.generate(**inputs, max_length=500)
        decoded = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
        
        d['translate'] = decoded
        fopenl.write(f'{json.dumps(d)}\n')

100%|██████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [37:09<00:00,  4.49it/s]
